In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
mnist_bldr = tfds.builder('mnist')
mnist_bldr.download_and_prepare()
datasets = mnist_bldr.as_dataset(shuffle_files = False)
mnist_train_orig = datasets['train']
mnist_test_orig = datasets['test']

In [2]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64
NUM_EPOCS = 20

In [3]:
mnist_train = mnist_train_orig.map(lambda item: (tf.cast(item['image'], tf.float32)/255.0,
                                                tf.cast(item['label'], tf.int32)))
mnist_test = mnist_test_orig.map(lambda item: (tf.cast(item['image'], tf.float32)/255.0,
                                                tf.cast(item['label'], tf.int32)))

In [4]:
tf.random.set_seed(1)
mnist_train = mnist_train.shuffle(buffer_size = BUFFER_SIZE,
                                 reshuffle_each_iteration = False)

In [5]:
mnist_valid = mnist_train.take(10000).batch(BATCH_SIZE)
mnist_train = mnist_train.skip(10000).batch(BATCH_SIZE)

In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters = 32,
                                kernel_size = (5,5),
                                strides = (1,1),
                                padding = 'same',
                                data_format = 'channels_last',
                                name = 'conv_1', activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2), name = 'pool_1'))
model.add(tf.keras.layers.Conv2D(filters = 64,
                                kernel_size = (5,5),
                                strides = (1,1),
                                padding = 'same',
                                name = 'conv_2',
                                activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2), name = 'pool_2'))

In [7]:
# calculate size of feature map
model.compute_output_shape(input_shape = (16, 28, 28, 1))

TensorShape([16, 7, 7, 64])

In [8]:
# add dense layer for classifier
# flatten first
model.add(tf.keras.layers.Flatten())
model.compute_output_shape(input_shape = (16,28,28, 1))

TensorShape([16, 3136])

In [9]:
# add two dense layers with dropout layer in between
model.add(tf.keras.layers.Dense(units = 1024,
                              name = 'fc_l',
                              activation = 'relu'))
model.add(tf.keras.layers.Dropout(rate = 0.5))
model.add(tf.keras.layers.Dense(units = 10,
                               name = 'fc_2',
                               activation = 'softmax'))

In [10]:
tf.random.set_seed(1)

In [12]:
model.build(input_shape = (None, 28, 28, 1))
model.compile(optimizer = tf.keras.optimizers.Adam(),
             loss = tf.keras.losses.SparseCategoricalCrossentropy(),
             metrics = ['accuracy'])